# Runoff

In [13]:
import xarray as xr
import rioxarray
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.ops import unary_union
import regionmask
import csv
from shapely.geometry import Point
import calendar
import matplotlib.pyplot as plt
import pyet
import os

In [14]:
# OGGM datasets for three basins 2302, 2306, and 2309. Change depending on area of interest
ds1 = xr.open_dataset('basin_2302_run_hydro_gcm_from_2000_bc_2000_2019_CESM2-WACCM_ssp370.nc')
ds2 = xr.open_dataset('basin_2306_run_hydro_gcm_from_2000_bc_2000_2019_CESM2-WACCM_ssp370.nc')
ds3 = xr.open_dataset('basin_2309_run_hydro_gcm_from_2000_bc_2000_2019_CESM2-WACCM_ssp370.nc')

ds_oggm = xr.concat([ds1, ds2, ds3], dim='rgi_id')  # Replace 'new_dim' with the appropriate dimension

shapefile_dir = 'basin_shapefile_folder'

# Read the CSV file into a DataFrame. Statistics file determined by region, found here (https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.3/elev_bands/W5E5/RGI62/b_080/L5/summary/)
df = pd.read_csv('glacier_statistics_13_14_15.csv')

# Load the shapefile
#shapefile = gpd.read_file('Indus_sub_sub_basin_118.shp')  # Replace with your shapefile path

# Output directory for the new NetCDF files
output_dir = 'r_output_folder'
os.makedirs(output_dir, exist_ok=True)

In [12]:
# Function to calculate average precipitation and save to a new NetCDF file
def process_shapefile(shapefile_path, ds_oggm):

    # Load the shapefile
    shapefile = gpd.read_file(shapefile_path)
    
    runoff = ds_oggm.runoff_monthly.squeeze()
    runoff = runoff.rename({'time':'year'})

    r_ts = runoff.stack(time=['year', 'month_2d'])  # transform 2d to 1D
    new_time = pd.to_datetime({'year': r_ts['year'], 'month': r_ts['month_2d'], 'day': 1})  # convert to standard calendar time
    r_ts['time'] = new_time.values

    ### OGGM, finding days / month

    time = r_ts['time']
    time_index = pd.to_datetime(time.values)

    # Calculate the differences between consecutive time steps
    time_diffs = time_index.to_series().diff().dropna()
    
    # Convert the differences to seconds
    days_per_month = time_diffs.dt.days

    # Convert the pandas Series to a numpy array
    days_array = days_per_month.values

    # Create a new DataArray for the seconds
    days_da = xr.DataArray(days_array, coords=[time_index[1:]], dims=['time'], name='days_per_month')

    # Add the new variable to the dataset (r_ts = the sum of the entire basin (will need to sum the galciers within each individual basin before this process)
    r_ts['days_per_month'] = days_da

    ### OGGM runoff / day, runoff monthly / days_per_month

    r_ts_per_day = r_ts / days_da #runoff for all glaciers in the file

    ### OGGM surface area of basin/subbasin
    ### Selecting glaciers within basin from a csv using shp file

    # Create a GeoDataFrame from the DataFrame
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.terminus_lon, df.terminus_lat))

    # Ensure the GeoDataFrame and the shapefile have the same CRS (Coordinate Reference System)
    gdf = gdf.set_crs(shapefile.crs, allow_override=True)

    # Perform the spatial join
    points_within_shapefile = gpd.sjoin(gdf, shapefile, how='inner', op='within')

    ### Extracting list of RGI_id's from csv and using it to select glaciers from the .nc file

    glaciers_within_shapefile_list = points_within_shapefile['rgi_id'].tolist()

    ### Attempt to find out which glaciers from the CSV/shapefile list are within the .nc file (which has three basins)

    rgi_ids_in_nc = r_ts['rgi_id'].values

    # Convert the NetCDF rgi_id tags to a set for faster lookup
    rgi_ids_in_nc_set = set(rgi_ids_in_nc)

    # Find the rgi_id tags from the list that are not in the NetCDF file
    missing_rgi_ids = [rgi_id for rgi_id in glaciers_within_shapefile_list if rgi_id not in rgi_ids_in_nc_set]

    # Find the rgi_id tags from the list that are in the NetCDF file
    present_rgi_ids = [rgi_id for rgi_id in glaciers_within_shapefile_list if rgi_id in rgi_ids_in_nc_set]

    list_length = len(present_rgi_ids)
    list_length = len(missing_rgi_ids)

    ### This number makes sense when you minus the .nc list from the CSV/shp list 

    list_length = len(missing_rgi_ids)

    # Select the glaciers based on present rgi_id tags
    selected_glaciers = r_ts.sel(rgi_id=present_rgi_ids) #FINALLY but no idea where the missing 2843 glaciers are...

    ### Correcting units
    ### Finding daily value of surface runoff (runoff kg/day)

    days_per_month_coord = selected_glaciers.coords['days_per_month']

    days_per_month_coord_broadcasted = days_per_month_coord.broadcast_like(selected_glaciers)

    selected_glaciers_runoff_daily = selected_glaciers/days_per_month_coord_broadcasted

    selected_glaciers_runoff_daily_ts = selected_glaciers_runoff_daily.sum(dim='rgi_id')

    ### Finding daily value per m2

    gdf = shapefile

    #gdf = gdf.to_crs(epsg=32633)

    gdf['area_m2'] = gdf['geometry'].area

    # Define the surface area value in square meters
    value = gdf['area_m2'].values[0]
    length = len(selected_glaciers_runoff_daily_ts['time'])

    # Create the DataArray
    Indus_glacier_surface_area = xr.DataArray(np.full(length, value), dims=['time'], name='constant_value')

    r_day_m2 = selected_glaciers_runoff_daily / Indus_glacier_surface_area

    r_day_m2_ts = r_day_m2.sum(dim='rgi_id')

    ### Adding them all together

    r_ts_crop_sub = r_day_m2_ts.sel(time=slice('2015-01-01T00:00:00.000000000', '2100-12-01T00:00:00.000000000'))

    # Save the new dataset to a NetCDF file
    output_file = os.path.join(output_dir, os.path.splitext(os.path.basename(shapefile_path))[0] + '_avg_runoff.nc')
    r_ts_crop_sub.to_netcdf(output_file)

    # Iterate over all shapefiles in the directory
for shapefile in os.listdir(shapefile_dir):
    if shapefile.endswith('.shp'):
        shapefile_path = os.path.join(shapefile_dir, shapefile)
        
        # Process each shapefile
        process_shapefile(shapefile_path, ds_oggm)


/Users/tomveale/anaconda3/envs/dissenv/lib/python3.12/site-packages/xarray/core/coordinates.py:41: FutureWarning: Updating MultiIndexed coordinate 'time' would corrupt indices for other variables: ['year', 'month_2d']. This will raise an error in the future. Use `.drop_vars({'time', 'year', 'month_2d'})` before assigning new coordinate values.
  self.update({key: value})
/Users/tomveale/anaconda3/envs/dissenv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/tomveale/anaconda3/envs/dissenv/lib/python3.12/site-packages/xarray/core/coordinates.py:41: FutureWarning: Updating MultiIndexed coordinate 'time' would corrupt indices for other variables: ['year', 'month_2d']. This will raise an error in the future. Use `.drop_vars({'time', 'year', 'month_2d'})` before assigning new coord